<a href="https://colab.research.google.com/github/jyonalee/Insider-Threat-and-Anomaly-Detection-from-User-Activities/blob/master/Anomaly_Detection_LSTM_Data_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install awscli to download the data
!pip3 install awscli --upgrade --user

# download data and save it on `data`
!mkdir data
!~/.local/bin/aws s3 sync --no-sign-request --region us-west-1 "s3://cse-cic-ids2018/Processed Traffic Data for ML Algorithms/" data/.

# Anomaly Detection with LSTM in Network Traffic Data

This project explores anomaly detection in network traffic with RNN-LSTM to train the model.

The dataset can be obtained [here](https://www.unb.ca/cic/datasets/ids-2018.html)

This is part of the capstone project for the Machine Learning Nano Degree from Udacity

## Data Exploration

---



In [1]:
import pandas as pd
import numpy as np
import os
import glob

In [2]:
# load data and do preliminary cleaning
directory = '/home/jlee/cse-cic-ids2018/Processed Traffic Data for ML Algorithms'

filepath = os.path.join(directory,'Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv')
df1 = pd.read_csv(filepath)
df1 = df1[df1['Protocol'] != 'Protocol']
# filepath = os.path.join(directory,'Friday-16-02-2018_TrafficForML_CICFlowMeter.csv')
# df2 = pd.read_csv(filepath)
# df2 = df2[df2['Protocol'] != 'Protocol']
filepath = os.path.join(directory,'Friday-02-03-2018_TrafficForML_CICFlowMeter.csv')
df3 = pd.read_csv(filepath)
# filepath = os.path.join(directory,'Friday-23-02-2018_TrafficForML_CICFlowMeter.csv')
# df4 = pd.read_csv(filepath)
# filepath = os.path.join(directory,'Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv')
# df5 = pd.read_csv(filepath)
# filepath = os.path.join(directory,'Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv')
# df6 = pd.read_csv(filepath)
# filepath = os.path.join(directory,'Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv')
# df7 = pd.read_csv(filepath)
filepath = os.path.join(directory,'Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv')
df8 = pd.read_csv(filepath)
df8 = df8[df8['Protocol'] != 'Protocol']
# filepath = os.path.join(directory,'Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv')
# df9 = pd.read_csv(filepath)


/home/jlee/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/jlee/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# filepath = os.path.join(directory,'Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv')
# df10 = pd.read_csv(filepath)
# df10 = df10[df10['Protocol'] != 'Protocol']

In [4]:
df = pd.concat([df1,df3,df8], ignore_index=True)

In [5]:
# clean up intermediary dataframes to free memory
del df1
# del df2
del df3
# del df4
# del df5
# del df6
# del df7
del df8
# del df9


In [6]:
len(df)

1992746

In [7]:
# get count of each label
print(df['Label'].value_counts())

Benign           1544621
Bot               286191
Infilteration     161934
Name: Label, dtype: int64


In [8]:
# get distribution in of each label
print(df['Label'].value_counts()/len(df))

Benign           0.775122
Bot              0.143616
Infilteration    0.081262
Name: Label, dtype: float64


so in essence, 73.8% of data points in this dataset is 'Benign' while the rest are some form of malicious attack

In [9]:
df.head()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,0,01/03/2018 08:17:11,115307855,5,0,0,0,0,0,...,0,1812348,0,1812348,1812348,56700000,6010057.622,61000000,52500000,Benign
1,0,0,01/03/2018 08:20:07,60997457,2,0,0,0,0,0,...,0,0,0,0,0,61000000,0,61000000,61000000,Benign
2,67,17,01/03/2018 08:17:18,61149019,5,0,1500,0,300,300,...,8,3530939,0,3530939,3530939,19200000,12500000,32600000,7999725,Benign
3,0,0,01/03/2018 08:22:09,60997555,2,0,0,0,0,0,...,0,0,0,0,0,61000000,0,61000000,61000000,Benign
4,0,0,01/03/2018 08:24:11,61997503,3,0,0,0,0,0,...,0,999909,0,999909,999909,61000000,0,61000000,61000000,Benign


In [13]:
# convert columns to correct data types
df[['Dst Port','Protocol','Label']] = df[['Dst Port','Protocol','Label']].astype(str)
df[df.columns.difference(['Dst Port','Protocol','Timestamp','Label'])] = df[df.columns.difference(['Dst Port','Protocol','Timestamp','Label'])].apply(pd.to_numeric, errors='coerce')
df['Timestamp'] = pd.to_datetime(df.Timestamp, format="%d/%m/%Y %H:%M:%S")

In [19]:
df = df.sort_values(by=['Timestamp'])

In [20]:
df.head()

,index,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,1735335,3386,6,2018-02-28 01:00:00,529751,2,1,0,0.0,0,...,20,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,Benign
1,1602978,53,17,2018-02-28 01:00:01,154058,1,1,40,167.0,40,...,8,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,Benign
2,1747280,0,0,2018-02-28 01:00:01,112640736,3,0,0,0.0,0,...,0,0.0,0.0,0.0,0.0,56320368.0,18.384776,56320381.0,56320355.0,Benign
3,1589646,443,6,2018-02-28 01:00:01,782742,9,8,1818,3898.0,1078,...,20,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,Benign
10,1614474,443,6,2018-02-28 01:00:02,2250028,11,12,2008,4434.0,1078,...,20,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,Benign


In [ ]:
# todo:
# more stats
# plot timeline of events
# determine what is a normal sequence vs not normal sequence and visualize if possible

In [ ]:
# todo:
# with the definition of a `normal sequence`, need to transform & process the data accordingly before training with LSTM
# with the baseline algorithms for anomaly detection, should be fine to train with given data as-is